# 22/11 further development -- adding columns to DB

In [3]:
"""
This library provides a database framework for QMD/QML.

A Pandas dataframe is used as a database for running QMD, recording: 
  - model name (and name reordered alphabetically) 
  - Log Likelihood
  - Origin epoch
  - QML Class
  - Qubits acted on: which qubits have some operator acting on 
  - Root Node
  - Selected
  - Status

A separate database holds all information on individual models:
  - constituent operators (names and matrices) [i.e. those which are summed to give model]
  - total matrix
  - number of qubits (dimension)

The database is generated by the function launch_db. E.g. usage: 

  $ db, model_db, model_lists = DataBase.launch_db(gen_list=gen_list)

This returns: 

  - db: "running database", info on dlog likelihood, etc.
  - model_db: info on construction of model, i.e. constituent operators etc.
  - model_lists = list of lists containing alphabetised model names. When a new model is considered, it should be compared against models of identical dimension (number of qubits) by alhpabetical name. If the alphabetical name is found in, e.g. model_lists[3], it has already been considered and the QML should be terminated.


To fill the data base, a list of generators are passed to launch_db. 
These are strings corresponding to unique models, e.g. 'xTy' means pauli_x TENSOR_PROD pauli_y 
(see Naming_Convention.pdf). 
These names are used to generate instances of the operator class (defined here). 
This class computes, based on the name, what the constituent operator names, matrices, total matrix, etc.
of the given model are, and fills these values into the model_db. 

e.g. usage of operator: 
  $ name='xPyTz'
  $ test_op = operator(name)
  $ print(test_op.name)
  $ print(test_op.matrix)
  $ print(test_op.constituent_operators

"""


import numpy as np
import itertools as itr

import os as os
import sys as sys 
import pandas as pd
import warnings
import hashlib
sys.path.append(os.path.join("..","Libraries","QML_lib"))

import Evo as evo

global paulis_list
paulis_list = {'i' : np.eye(2), 'x' : evo.sigmax(), 'y' : evo.sigmay(), 'z' : evo.sigmaz()}



"""
------ ------ Operator Class ------ ------
"""

class operator():
    """
    Operator class:
    Takes one argument: name (string) according to naming convention.
    Name specifies all details of operator. 
    e.g.
    - xPy is X+Y, 1 qubit
    - xTz is x TENSOR_PROD Z, 2 qubits
    - xMyTz is (X PROD Y) TENSOR_PROD Z, 2 qubits
    - xPzTiTTz is (X+Z) TENSOR_PROD I TENSOR_PROD Z
      -- 3 qubit operator. X+Z on qubit 1; I on qubit 2; Z on qubit 3
    See Naming_Convention.pdf for details.

    Constituents of an operator are operators of the same dimension which sum to give the operator.
    e.g. 
    - xPy = X + Y has constituents X, Y

    Assigns properties for : 
    - constituents_names: strings specifying constituents
    - constituents_operators: whole matrices of constituents
    - num_qubits: total dimension of operator [number of qubits it acts on]
    - matrix: total matrix operator
    - qubits_acted_on: list of qubits which are acted on non-trivially
      -- e.g. xTiTTz has list [1,3], since qubit 2 is acted on by identity
    - alph_name: rearranged version of name which follows alphabetical convention
      -- uniquely identifies equivalent operators for comparison against previously considered models
    -
    """
    def __init__(self, name): 
        self.name = name
    
    @property
    def constituents_names(self):
        """
        List of constituent operators names.
        """
        t_str, p_str, max_t, max_p = get_t_p_strings(self.name)
        paulis_list = {'i' : np.eye(2), 'x' : evo.sigmax(), 'y' : evo.sigmay(), 'z' : evo.sigmaz()}
        if(max_t >= max_p):
            # if more T's than P's in name, it has only one constituent. 
            return [self.name]
        else: 
            # More P's indicates a sum at the highest dimension. 
            return self.name.split(p_str)

    @property
    def num_qubits(self):
        """
        Number of qubits this operator acts on. 
        """
        return get_num_qubits(self.name)
        
    @property
    def constituents_operators(self):
        """
        List of matrices of constituents. 
        """
        ops = []
        for i in self.constituents_names:
            ops.append(compute(i))
        return ops

    @property
    def num_constituents(self):
        """
        Integer, how many constituents, and therefore parameters, are in this model.
        """    
        return len(self.constituents_names)
    
    @property 
    def matrix(self):
        """
        Full matrix of operator. 
        """
        mtx = empty_array_of_same_dim(self.name)
        for i in self.constituents_operators:
            mtx += i
        return mtx

    @property
    def qubits_acted_on(self):
        """
        List of qubits which are acted on non-trivially by this operator. 
        TODO: qubit count starts from 1 -- should it start from 0?
        """
        return list_used_qubits(self.name)
   
    @property 
    def two_to_power_used_qubits_sum(self):
        """
        Binary sum of operators acted on. 
        For use in comparing new operators. [Not currently used]
        """
        running_sum = 0
        for element in list_used_qubits(self.name):
            running_sum += 2**element
        return running_sum

    @property
    def alph_name(self):
        """
        Name of operator rearranged to conform with alphabetical naming convention. 
        Uniquely identifies equivalent operators. 
        For use when comparing potential new operators. 
        """
        return alph(self.name)
    
    
"""
Functions for use by operator class to parse string (name) and prodcue relevent operators, lists etc.
"""

def get_num_qubits(name):
    """
    Parse string and determine number of qubits this operator acts on. 
    """
    max_t_found = 0 
    t_str=''
    while name.count(t_str+'T')>0:
        t_str=t_str+'T'

    num_qubits = len(t_str) + 1
    return num_qubits
    

def list_used_qubits(name):
    """
    Parse string and determine which qubits are acted on non-trivially. 
    """
    max_t, t_str = find_max_letter(name, "T")
    max_p, p_str = find_max_letter(name, "P")
    running_list = []

    if max_p >= max_t:
        list_by_p_sep = []
        if p_str == '':  
          ## In case of empty separator, split by anything into one string    
          p_str = 'RRR'
        
        sep_by_p = name.split(p_str)
        for element in sep_by_p:
            list_by_p_sep.append(get_acted_on_qubits(element))

        for i in range(len(list_by_p_sep)):
            to_add= list(set(list_by_p_sep[i]) - set(running_list))
            running_list = running_list + to_add

    else:
        running_list = get_acted_on_qubits(name)
    return running_list


def get_acted_on_qubits(name):
    """
    Parse string and determine which qubits are acted on non-trivially. 
    """
    max_t, t_str = find_max_letter(name, "T")
    max_p, p_str = find_max_letter(name, "P")
    if max_p > max_t:
        list_by_p_sep = []
        if p_str == '':
          ## In case of empty separator, split by anything into one string    
          p_str = 'RRR'

        sep_by_p = name.split(p_str)
        for element in sep_by_p:
            list_by_sep.append(fill_qubits_acted_on_list, element)
    
    
    qubits_acted_on = []
    fill_qubits_acted_on_list(qubits_acted_on,name)
    return sorted(qubits_acted_on)
    
def fill_qubits_acted_on_list(qubits_acted_on, name):
    """
    Parse string and determine which qubits are acted on non-trivially. 
    Return list of those qubits. 
    """
    max_t, t_str = find_max_letter(name, "T")
    max_p, p_str = find_max_letter(name, "P")
    if(max_p > max_t):
        string_to_analyse = name.split(p_str)[0]
    else:
        string_to_analyse = name

    if max_t == 0:
        if string_to_analyse != 'i':
            qubits_acted_on.append(1)


    else:
        i=max_t
        this_t_str = t_str
        broken_down = string_to_analyse.split(this_t_str)
        lhs = broken_down[0]
        rhs = broken_down[1]
        if rhs !='i':
            qubits_acted_on.append(i+1)

        if max_t == 1:
            if lhs!='i':
                qubits_acted_on.append(1)
        else: 
            fill_qubits_acted_on_list(qubits_acted_on, lhs)                
    
def get_t_p_strings(name):
    """
    Find largest instance of consecutive P's and T's.
    Return those instances and lengths of those instances. 
    """
    t_str = ''
    p_str = ''
    while name.count(t_str+'T')>0:
        t_str=t_str+'T'

    while name.count(p_str+'P')>0:
        p_str=p_str+'P'

    max_t = len(t_str)
    max_p = len(p_str)

    return t_str, p_str, max_t, max_p        
    
def find_max_letter(string, letter):
    """
    Find largest instance of consecutive given 'letter'.
    Return largest instance and length of that instance. 
    """
    letter_str=''
    while string.count(letter_str+letter)>0:
        letter_str=letter_str+letter

    return len(letter_str), letter_str


def empty_array_of_same_dim(name):
    """
    Parse name to find size of system it acts on. 
    Produce an empty matrix of that dimension and return it. 
    """
    t_str=''
    while name.count(t_str+'T')>0:
        t_str=t_str+'T'

    num_qubits = len(t_str) +1
    dim = 2**num_qubits
    #print("String: ", name, " has NQubits: ", num_qubits)
    empty_mtx = np.zeros([dim, dim], dtype=np.complex128)
    return empty_mtx



def alph(name):
    """
    Return alphabetised version of name. 
    Parse string and recursively call alph function to alphabetise substrings. 
    """
    t_max, t_str = find_max_letter(name, "T")
    p_max, p_str = find_max_letter(name, "P")
    m_max, m_str = find_max_letter(name, "M")
    
    if p_max == 0 and t_max ==0 and p_max ==0 :
        return name
    
    if p_max > t_max and p_max > m_max: 
        ltr = 'P'
        string = p_str
    elif t_max >= p_max:
        string = t_str
        ltr = 'T'
    elif m_max >= p_max: 
        string = m_str
        ltr = 'M'
    elif t_max > m_max: 
        string = t_str
        ltr = 'T'
    else:
        ltr = 'M'
        string = m_str

    spread = name.split(string)
    if  p_max==m_max and p_max > t_max:
        string = p_str
        list_elements = name.split(p_str)
        
        for i in range(len(list_elements)):
            list_elements[i] = alph(list_elements[i])
        sorted_list = sorted(list_elements)
        linked_sorted_list = p_str.join(sorted_list)
        return linked_sorted_list
        
    if ltr=='P' and p_max==1:
        sorted_spread = sorted(spread)
        out = string.join(sorted_spread)
        return out
    elif ltr=='P' and p_max>1:
        list_elements = name.split(string)
        sorted_list = sorted(list_elements)
        for i in range(len(sorted_list)):
            sorted_list[i] = alph(sorted_list[i])
        linked_sorted_list = string.join(sorted_list)
        return linked_sorted_list
    else: 
        for i in range(len(spread)):
            spread[i] = alph(spread[i])
        out = string.join(spread)
        return out


def compute_t(inp):
    """
    Assuming largest instance of action on inp is tensor product, T.
    Parse string.
    Recursively call compute() function.
    Tensor product resulting lists.
    Return operator which is specified by inp.
    """
    max_t, t_str = find_max_letter(inp, "T")
    max_p, p_str = find_max_letter(inp, "P")

    if(max_p == 0 and max_t==0):
        pauli_symbol = inp
        return paulis_list[pauli_symbol] 

    elif(max_t==0):
        return compute(inp)
    else:
        to_tens = inp.split(t_str)
        #print("To tens: ", to_tens)
        running_tens_prod=compute(to_tens[0])
        #print("Split by ", t_str, " : \n", to_tens)
        for i in range(1,len(to_tens)):
            max_p, p_str = find_max_letter(to_tens[i], "P")
            max_t, t_str = find_max_letter(to_tens[i], "T")
            #print("To tens [i=", i, "]:\n", to_tens[i] )
            rhs = compute(to_tens[i])
            running_tens_prod = np.kron(running_tens_prod, rhs)
        #print("RESULT ", t_str, " : ", inp, ": \n", running_tens_prod)
        return running_tens_prod

def compute_p(inp):
    """
    Assuming largest instance of action on inp is addition, P.
    Parse string.
    Recursively call compute() function.
    Sum resulting lists.
    Return operator which is specified by inp.
    """
    max_p, p_str = find_max_letter(inp, "P")
    max_t, t_str = find_max_letter(inp, "T")

    if(max_p == 0 and max_t==0):
        pauli_symbol = inp
        return paulis_list[pauli_symbol] 

    elif max_p==0:
        return compute(inp)
    else: 
        to_add = inp.split(p_str)
        #print("To add : ", to_add)
        running_sum = empty_array_of_same_dim(to_add[0])
        for i in range(len(to_add)):
            max_p, p_str = find_max_letter(to_add[i], "P")
            max_t, t_str = find_max_letter(to_add[i], "T")

           # print("To add [i=", i, "]:", to_add[i] )
            rhs = compute(to_add[i])
            #print("SUM shape:", np.shape(running_sum))
            #print("RHS shape:", np.shape(rhs))
            running_sum += rhs

        #print("RESULT ", p_str, " : ", inp, ": \n", running_sum)
        return running_sum


def compute_m(inp):
    """
    Assuming largest instance of action on inp is multiplication, M.
    Parse string.
    Recursively call compute() function.
    Multiple resulting lists.
    Return operator which is specified by inp.
    """

    max_m, m_str = find_max_letter(inp, "M")
    max_p, p_str = find_max_letter(inp, "P")
    max_t, t_str = find_max_letter(inp, "T")

    if(max_m == 0 and max_t==0 and max_p == 0 ):
        pauli_symbol = inp
        return paulis_list[pauli_symbol] 

    elif max_m ==0:
        return compute(inp)
    
    else:   
        to_mult = inp.split(m_str)
        #print("To mult : ", to_mult)
        t_str=''
        while inp.count(t_str+'T')>0:
            t_str=t_str+'T'

        num_qubits = len(t_str) +1
        dim = 2**num_qubits

        running_product = np.eye(dim)

        for i in range(len(to_mult)):
            running_product = np.dot(running_product, compute(to_mult[i]))

        return running_product    
    
def compute(inp):
    """
    Parse string.
    Recursively call compute() functions (compute_t, compute_p, compute_m).
    Tensor product, multiply or sum resulting lists.
    Return operator which is specified by inp.
    """

    max_p, p_str = find_max_letter(inp, "P")
    max_t, t_str = find_max_letter(inp, "T")
    max_m, m_str = find_max_letter(inp, "M")

    if(max_m == 0 and max_t==0 and max_p == 0):
        pauli_symbol = inp
        return paulis_list[pauli_symbol] 
    elif max_m > max_t:
        return compute_m(inp)
    elif max_t >= max_p:
        return compute_t(inp)
    else:
        return compute_p(inp)    

/usr/local/lib/python2.7/dist-packages/IPython/parallel.py:13: ShimWarning: The `IPython.parallel` package has been deprecated since IPython 4.0. You should import from ipyparallel instead.
  "You should import from ipyparallel instead.", ShimWarning)
/usr/local/lib/python2.7/dist-packages/qinfer/parallel.py:52: UserWarning: Could not import IPython parallel. Parallelization support will be disabled.
  "Could not import IPython parallel. "


In [ ]:
from qinfer import NormalDistribution
a=NormalDistribution(mean=0.5, var=0.15)  

In [ ]:
a

In [4]:
"""
------ ------ Database declaration and functions ------ ------
"""

def launch_db(RootN_Qbit=[0], N_Qubits=1, gen_list=[]):
    """
    Inputs:
    TODO
    RootN_Qbit: TODO
    N_Qubits: TODO
    gen_list: list of strings corresponding to model names. 
    
    Outputs: 
      - db: "running database", info on log likelihood, etc.
      - model_db: info on construction of model, i.e. constituent operators etc.
      - model_lists = list of lists containing alphabetised model names. When a new model is considered, it     

    Usage: 
        $ gen_list = ['xTy, yPz, iTxTTy] # Sample list of model names
        $ running_db, model_db, model_lists = DataBase.launch_db(gen_list=gen_list)
    
    """
    generators = []
    total_model_list = []
#     TODO: Is this the absolute total ever???? Could define this as a global variable at top of file. 
    Max_N_Qubits = 13
    model_lists = {}
    for j in range(1, Max_N_Qubits):
        model_lists[j] = []
    
    for i in gen_list:
        generators.append(operator(i))
        alph_model_name = alph(i)
        num_qubits = get_num_qubits(i)
        model_lists[num_qubits].append(alph_model_name)

    model_db = pd.DataFrame({
        '<Name>' : [ gen.name for gen in generators], 
        'Alph_Name' :[ gen.alph_name for gen in generators],
        'All_Operators_Names': [gen.constituents_names for gen in generators],
        'All_Operators_Matrices': [gen.constituents_operators for gen in generators],
        'N_params' : [ gen.num_constituents for gen in generators ],
        'Num_Qubits' : [ gen.num_qubits for gen in generators ],
        'Binary_Sum_Used_Qubits' : [ gen.two_to_power_used_qubits_sum for gen in generators ],
        'Matrix' : [gen.matrix for gen in generators]
    })
        
    # if N_qubits defined: work out generator list.
    # Or should number qubits be implied by gen list?
    db = pd.DataFrame({
        'Model_Class_Instance' : [gen for gen in generators],
        '<Name>' : [ gen.name for gen in generators], 
        'Alph_Name' :[ gen.alph_name for gen in generators],
        'Qubits_Acted_On' : [ gen.qubits_acted_on for gen in generators ],
        'DB_location' : [ get_location(model_db, gen.name) for gen in generators],
        'Status' : 'Ready', #TODO can get rid?
        'Selected' : False, #TODO what's this for?
        'TreeID' : [0 for gen in generators ], # TODO proper tree id's,
        'Param_Estimates' : [ [[None, None]]*gen.num_constituents for gen in generators ]
        
#        'LogL_Ext' : None, 
#        'QML_Class' : None, 
#        'Origin_epoch' : 0, 
#        'RootNode' : 'NaN',
        })  
        
    return db, model_db, model_lists


def add_model(model_name, running_database, model_db, model_lists ):
    """
    Function to add a model to the existing databases. 
    First checks whether the model already exists. 
    If so, does not add model to databases.
      TODO: do we want to return False in this case and use as a check in QMD?
    
    Inputs: 
      - model_name: new model name to be considered and added if new. 
      - running_database: Database (output of launch_db) containing info on log likelihood etc. 
      - model_db: Database (output of launch_db) containing info on models, e.g constituent info, etc. 
      - model_lists: output of launch_db. A list of lists containing every previously considered model, categorised by dimension. 
      
    Outputs: 
      TODO: return True if added; False if previously considered? 
      
    Effect: 
      - If model hasn't been considered before, 
          Adds a row to running_database and model_db containing all columns of those.     
    """    
    
    alph_model_name = alph(model_name)
    model_num_qubits = get_num_qubits(model_name)
    
    if consider_new_model(model_lists, model_name, running_database)== 'New':
        print("Model Not previously considered -- adding")
        op = operator(model_name)
        num_rows = len(running_database)

        # add model_db_new_row to model_db and running_database
        # Note: do NOT use pd.df.append() as this copies total DB,
        # appends and returns copy.
        
        model_db_new_row = pd.Series({
            '<Name>': op.name, 
            'Model_Class_Instance' : gen,
            'Alph_Name': op.alph_name,
            'All_Operators_Names' : op.constituents_names,
            'All_Operators_Matrices': op.constituents_operators,
            'N_params' : op.num_constituents,
            'Num_Qubits' : op.num_qubits,
            'Binary_Sum_Used_Qubits' : op.two_to_power_used_qubits_sum,
            'Matrix' : op.matrix
            })

        model_db.loc[num_rows] = model_db_new_row
       # total_model_list.append(alph_model_name)
        model_lists[model_num_qubits].append(alph_model_name)
        
        # Add to running_database too, after adding to model_db
        
        running_db_new_row = pd.Series({
            '<Name>': op.name,
            'Alph_Name' : op.alph_name,
            'Qubits_Acted_On' : op.qubits_acted_on,
            'DB_location' : get_location(model_db, model_name),
            'Status' : 'Ready', 
            'Selected' : False, 
            'TreeID' : 0, #TODO make argument of add_model fnc,
            'Param_Estimates' : [[None, None]]*op.num_constituents
#            'LogL_Ext' : None, 
#            'QML_Class' : None, 
#            'Origin_epoch' : 0, 
#            'RootNode' : 'NaN',
        })

        running_database.loc[num_rows] = running_db_new_row      
        
    else:
        location = consider_new_model(model_lists, model_name, running_database)
        #db_loc = get_location_by_alph_name(model_db, model_name)
        print("Model", alph_model_name, " previously considered at location", location)  



def get_location(db, name):
    """
    Return which row in db corresponds to the string name.
    """
    for i in range(len(db['<Name>'])):
        if db['<Name>'][i] == name:
            return i

def get_location_by_alph_name(db, name):
    """
    Return which row in db corresponds to the string name.
    Pass in alphabetised version of name. 
    
    """
    for i in range(len(db['Alph_Name'])):
        if db['Alph_Name'][i] == name:
            return i

        
def consider_new_model(model_lists, name, db):
    """
    Check whether the new model, name, exists in all previously considered models, 
    held in model_lists. 
    If name has not been previously considered, 'New' is returned. 
    If name has been previously considered, the corresponding location in db is returned. 
    TODO: return something else? Called in add_model function. 
    """
    # Return true indicates it has not been considered and so can be added
    al_name = alph(name)
    n_qub = get_num_qubits(name)
    if al_name not in model_lists[n_qub]:
        return 'New'
    else: 
        location = get_location_by_alph_name(db, al_name)
        return location


        

In [5]:
gen_list = ['x', 'xTyPPyTz', 'xPyPzPi']
db, model_db, model_lists = launch_db(gen_list=gen_list)

In [6]:
db

,<Name>,Alph_Name,DB_location,Model_Class_Instance,Param_Estimates,Qubits_Acted_On,Selected,Status,TreeID
0,x,x,0,<__main__.operator instance at 0x7f2304a3f248>,"[[None, None]]",[1],False,Ready,0
1,xTyPPyTz,xTyPPyTz,1,<__main__.operator instance at 0x7f2304129b00>,"[[None, None], [None, None]]","[1, 2]",False,Ready,0
2,xPyPzPi,iPxPyPz,2,<__main__.operator instance at 0x7f2304129c20>,"[[None, None], [None, None], [None, None], [No...",[1],False,Ready,0


In [ ]:
add_model('yTzPPzTy', running_database=db, model_db=model_db, model_lists=model_lists)

In [ ]:
db

In [ ]:
db.loc[1]["Param_Estimates"][0]

# Before 22/11/17 development below [legacy]

In [ ]:
import numpy as np
import itertools as itr

import os as os
import sys as sys 
import pandas as pd
import warnings
sys.path.append(os.path.join("..","Libraries","QML_lib"))
import Evo as evo

In [ ]:
global mycolumns
mycolumns = ['<Name>' , 'N_Qbit' , 'N_params' , 'Status' , 'Selected' , 'LogL_Ext' , 'QML_Class' , 'Origin_epoch' ,'All_Operators']

global spinnames
spinnames  = ['x', 'y', 'z']

global MaxParamRate
MaxParamRate = 3

global paulis_list
paulis_list = {'i' : np.eye(2), 'x' : evo.sigmax(), 'y' : evo.sigmay(), 'z' : evo.sigmaz()}

paulilist = evo.paulilist()


In [ ]:
dtb = pd.DataFrame
({
    '<Name>':'test',
    
})

In [ ]:
def generate_operators(generators):
    print("Number of generators: ", len(generators))

    for i in range(len(generators)):
        print(i, "\n", generators[i])
    for i in range(0,len(generators)):
        op_i = generators[i]
        for j in range(i+1,len(generators)):
            op_i = generators[i]        
            print("i,j = ", i,j)
            print("op_0: \n", op_0)
            print("op_1: \n", op_1)

In [ ]:
generate_operators(generators)

In [ ]:
generators = list()
generators_names = list()

generators.append(evo.sigmay())
generators.append(evo.sigmax())
generators.append(evo.sigmaz())
generators_names.append('y')
generators_names.append('x')
generators_names.append('z')


In [ ]:
operators = pd.DataFrame({
    'Name':['i', 'x', 'y', 'z'],
    'Op_list':[np.identity(2), evo.sigmax(), evo.sigmay(), evo.sigmaz()]
            })

In [ ]:
operators.loc[1,'Name']

In [ ]:
dd = 'xty01'

In [ ]:
len(dd)

In [ ]:
global spinnames
spinnames  = ['x', 'y', 'z']


In [ ]:
for spinname in spinnames:
    print(spinname+str(0))

In [ ]:
RootN_Qbit = [0]
x = evo.sigmax()
y = evo.sigmay()
z = evo.sigmaz()

xTx = np.kron(x,x)
xTy = np.kron(x,y)

In [ ]:
class operator():
    # Better to have this info in a DB than recalculating when called?
    def __init__(self, name): 
        self.name = name
    
    @property
    def constituents_names(self):
        t_str, p_str, max_t, max_p = get_t_p_strings(self.name)
        paulis_list = {'i' : np.eye(2), 'x' : evo.sigmax(), 'y' : evo.sigmay(), 'z' : evo.sigmaz()}
        if(max_t >= max_p):
            #constituent_names.append(self.name)
            return [self.name]
        else: 
            return self.name.split(p_str)
            #constituent_names = self.name.split(p_str)
        #return constituent_names        

    @property
    def num_qubits(self):
        max_t_found = 0 
        t_str=''
        while self.name.count(t_str+'T')>0:
            t_str=t_str+'T'

        num_qubits = len(t_str) + 1
        return num_qubits

    @property
    def constituents_operators(self):
        ops = []
        for i in self.constituents_names:
            ops.append(compute(i))
        return ops

    @property
    def num_constituents(self):
        # 1 param per constituent?
        return len(self.constituents_names)
    
    #THis is giving problems
    @property 
    def matrix(self):
        mtx = empty_array_of_same_dim(self.name)
        for i in self.constituents_operators:
            mtx += i
        return mtx
    
    def test_const(self):
        for i in self.constituents_operators:
            print(i)
            
    def get_t_p_strings(self):
        t_str = ''
        p_str = ''
        while self.name.count(t_str+'T')>0:
            t_str=t_str+'T'

        while self.name.count(p_str+'P')>0:
            p_str=p_str+'P'

        max_t = len(t_str)
        max_p = len(p_str)

        return t_str, p_str, max_t, max_p        
    
    def interactions_on(self):
        # work out which qubits have actions; list ?
        return 0

    def find_max_letter(string, letter):
        letter_str=''
        while string.count(letter_str+letter)>0:
            letter_str=letter_str+letter

        return len(letter_str), letter_str


    def empty_array_of_same_dim(name):
        t_str=''
        while name.count(t_str+'T')>0:
            t_str=t_str+'T'

        num_qubits = len(t_str) +1
        dim = 2**num_qubits
        #print("String: ", name, " has NQubits: ", num_qubits)
        empty_mtx = np.zeros([dim, dim], dtype=np.complex128)
        return empty_mtx

    def compute_t(inp):
       # print("Compute t : ", inp)
        max_t, t_str = find_max_letter(inp, "T")
        max_p, p_str = find_max_letter(inp, "P")

        if(max_p == 0 and max_t==0):
            pauli_symbol = inp
            return paulis_list[pauli_symbol] 

        elif(max_t==0):
            return compute(inp)
        else:
            to_tens = inp.split(t_str)
            running_tens_prod=compute(to_tens[0])
            #print("Split by ", t_str, " : \n", to_tens)
            for i in range(1,len(to_tens)):
                max_p, p_str = find_max_letter(to_tens[i], "P")
                max_t, t_str = find_max_letter(to_tens[i], "T")
                #print("To tens [i=", i, "]:\n", to_tens[i] )
                rhs = compute(to_tens[i])
                running_tens_prod = np.kron(running_tens_prod, rhs)
            #print("RESULT ", t_str, " : ", inp, ": \n", running_tens_prod)
            return running_tens_prod

    def compute_p(inp):
        #print("Compute p : ", inp)
        max_p, p_str = find_max_letter(inp, "P")
        max_t, t_str = find_max_letter(inp, "T")

        if(max_p == 0 and max_t==0):
            pauli_symbol = inp
            return paulis_list[pauli_symbol] 

        elif max_p==0:
            return compute_t(inp)
        else: 
            to_add = inp.split(p_str)
            #print("Split by ", p_str, "; len to_add ",len(to_add), ":\n", to_add)
            running_sum = empty_array_of_same_dim(to_add[0])
            for i in range(len(to_add)):
                max_p, p_str = find_max_letter(to_add[i], "P")
                max_t, t_str = find_max_letter(to_add[i], "T")

               # print("To add [i=", i, "]:", to_add[i] )
                rhs = compute(to_add[i])
                #print("SUM shape:", np.shape(running_sum))
                #print("RHS shape:", np.shape(rhs))
                running_sum += rhs

            #print("RESULT ", p_str, " : ", inp, ": \n", running_sum)
            return running_sum

    def compute(inp):
        #print("Computing ", inp)
        max_p, p_str = find_max_letter(inp, "P")
        max_t, t_str = find_max_letter(inp, "T")

        if(max_p == 0 and max_t==0):
            pauli_symbol = inp
            return paulis_list[pauli_symbol] 

        elif max_t >= max_p:
           # print("Max t=", max_t, ">= max p=", max_p)
            return compute_t(inp)
        else:
            return compute_p(inp)    
    

In [ ]:
def get_t_p_strings(name):
    t_str = ''
    p_str = ''
    while name.count(t_str+'T')>0:
        t_str=t_str+'T'

    while name.count(p_str+'P')>0:
        p_str=p_str+'P'

    max_t = len(t_str)
    max_p = len(p_str)

    return t_str, p_str, max_t, max_p        


In [ ]:
get_t_p_strings('xPxPxTTy')

In [ ]:
gen1 = operator('yTxPPzTi')
gen2 = operator('xTyPPyTiTTiTyPPxTiPPPxTyPPyTiTTiTyPPxTi')
gen3 = operator('xTy')

In [ ]:
gen1.constituents_names

gen1.operator

In [ ]:
gen1

In [ ]:
tst = 'xPyTzTTyPPPiTiPyTTz'
res_fnc=compute(tst)

In [ ]:
np.shape(a)

In [ ]:
a=x+y
b=np.kron(a,z)
c=np.kron(b,y)
d=i+y
e= np.kron(i,d)
f=np.kron(e,z)
res = c+f

In [ ]:
np.max(np.abs(res-res_fnc))

In [ ]:
gen3.constituents_operators

In [ ]:
if len(ls[2]):
    print("this")
else:
    print("that")

In [ ]:
generators_list = [gen1, gen2, gen3]

In [ ]:
db_test = Init_N_Qubit_DB(generators=generators_list)

In [ ]:
db_test

In [ ]:
### Idea for N qubit DB initialisation
def Init_N_Qubit_DB(RootN_Qbit=[0], N_Qubits=1, generators = paulilist):
    # if N_qubits defined: work out generator list.
    # Or should number qubits be implied by gen list?
    db = pd.DataFrame({
        '<Name>' : [   [gen.name] for gen in generators], 
        'N_Qbit' : [RootN_Qbit for i in range(len(generators))],
        'N_params' : [ gen.num_constituents for gen in generators ],
        'Num_Qubits' : [ gen.num_qubits for gen in generators ],
        'Status' : 'Ready', 
        'Selected' : False, 
        'LogL_Ext' : None, 
        'QML_Class' : None, 
        'Origin_epoch' : 0, 
        'RootNode' : 'NaN',
        'All_Operators_Names': [gen.constituents_names for gen in generators],
        'All_Operators_Matrices': [gen.constituents_operators for gen in generators]
        })  
        
    return db

In [ ]:
def InitialiseDB(RootN_Qbit = [0]):
    db = pd.DataFrame({
        '<Name>' : [   [spinname+str(RootN_Qbit[0])] for spinname in spinnames], 
        'N_Qbit' : [RootN_Qbit for i in range(len(spinnames))],
        'N_params' : 1, 
        'Status' : 'Ready', 
        'Selected' : False, 
        'LogL_Ext' : None, 
        'QML_Class' : None, 
        'Origin_epoch' : 0, 
        'RootNode' : 'NaN',
        'All_Operators': list(map(lambda j: np.array([evo.paulilist()[j]]) , 
                        range(  len(evo.paulilist())  ))) 
        })  
        
    return db


In [ ]:
name='xPyPzPiTxPyPzPiPPxPyPzPiTxPyPzPi'
num_levels_p = len(p_str)
num_terms=1+name.count(p_str)
p_array = np.empty([num_levels_p, num_terms])

In [ ]:
def compute_1p(inp):
    to_add = inp.split("P")
    running_sum = empty_array_of_same_dim(inp)
    for i in range(len(to_add)):
        pauli_symbol = to_add[i]
        rhs = paulis_list[pauli_symbol]
        running_sum += rhs
    return running_sum

In [ ]:
def compute_1t(inp):
    to_tens = inp.split("T")
    for i in range(len(to_tens)):
        to_tens[i] = compute_1p(to_tens[i])
    
    running_tens_prod = to_tens[0]
    for i in range(1,len(to_tens)):
        running_tens_prod = np.kron(running_tens_prod, to_tens[i])
    return running_tens_prod

In [ ]:
compute_1p('xPy')

In [ ]:
compute_p('xTy',1)

In [ ]:
compute_t('xTx')

In [ ]:
x = evo.sigmax()
y = evo.sigmay()
z = evo.sigmaz()

In [ ]:
def find_max_letter(string, letter):
    letter_str=''
    while string.count(letter_str+letter)>0:
        letter_str=letter_str+letter

    return len(letter_str), letter_str


def empty_array_of_same_dim(name):
    t_str=''
    while name.count(t_str+'T')>0:
        t_str=t_str+'T'

    num_qubits = len(t_str) +1
    dim = 2**num_qubits
    #print("String: ", name, " has NQubits: ", num_qubits)
    empty_mtx = np.zeros([dim, dim], dtype=np.complex128)
    return empty_mtx

def compute_t(inp):
   # print("Compute t : ", inp)
    max_t, t_str = find_max_letter(inp, "T")
    max_p, p_str = find_max_letter(inp, "P")

    if(max_p == 0 and max_t==0):
        pauli_symbol = inp
        return paulis_list[pauli_symbol] 
    
    elif(max_t==0):
        return compute(inp)
    else:
        to_tens = inp.split(t_str)
        running_tens_prod=compute(to_tens[0])
        #print("Split by ", t_str, " : \n", to_tens)
        for i in range(1,len(to_tens)):
            max_p, p_str = find_max_letter(to_tens[i], "P")
            max_t, t_str = find_max_letter(to_tens[i], "T")
            #print("To tens [i=", i, "]:\n", to_tens[i] )
            rhs = compute(to_tens[i])
            running_tens_prod = np.kron(running_tens_prod, rhs)
        #print("RESULT ", t_str, " : ", inp, ": \n", running_tens_prod)
        return running_tens_prod

def compute_p(inp):
    #print("Compute p : ", inp)
    max_p, p_str = find_max_letter(inp, "P")
    max_t, t_str = find_max_letter(inp, "T")
    
    if(max_p == 0 and max_t==0):
        pauli_symbol = inp
        return paulis_list[pauli_symbol] 

    elif max_p==0:
        return compute_t(inp)
    else: 
        to_add = inp.split(p_str)
        #print("Split by ", p_str, "; len to_add ",len(to_add), ":\n", to_add)
        running_sum = empty_array_of_same_dim(to_add[0])
        for i in range(len(to_add)):
            max_p, p_str = find_max_letter(to_add[i], "P")
            max_t, t_str = find_max_letter(to_add[i], "T")

           # print("To add [i=", i, "]:", to_add[i] )
            rhs = compute(to_add[i])
            #print("SUM shape:", np.shape(running_sum))
            #print("RHS shape:", np.shape(rhs))
            running_sum += rhs

        #print("RESULT ", p_str, " : ", inp, ": \n", running_sum)
        return running_sum

def compute(inp):
    #print("Computing ", inp)
    max_p, p_str = find_max_letter(inp, "P")
    max_t, t_str = find_max_letter(inp, "T")
    
    if(max_p == 0 and max_t==0):
        pauli_symbol = inp
        return paulis_list[pauli_symbol] 
    
    elif max_t >= max_p:
       # print("Max t=", max_t, ">= max p=", max_p)
        return compute_t(inp)
    else:
        return compute_p(inp)

In [ ]:
h='xPyTzPPzPxTy'
ham=compute(h)

In [ ]:
a=x+y
b=np.kron(a,z)
c=x+z
d=np.kron(c,y)
ham1=b+d
print(np.max(np.abs(ham1-ham)))

In [ ]:
#np.shape(operator_from_string('xTyPPyTzTTz'))
#np.shape(operator_from_string('xPyTiTTzPx'))
compute('xPyPiPzTiTTzPxPi')

In [ ]:
np.shape(compute('xTyTTTzTi'))

In [ ]:
a =np.kron(np.kron(x,y), np.kron(z,i))
d = compute('xTyTTTzTi')
print(np.max(np.abs(a-d)))

In [ ]:
np.shape(a)

In [ ]:
np.kron(x+y, y+y) + np.kron(x+y, y+y)

In [ ]:
a=np.kron(np.kron(y,i), z)

In [ ]:
b=operator_from_string('yTiTz')

In [ ]:
np.max(np.abs(a-b))

In [ ]:
a=operator_from_string('iPiTiTTz')

np.shape(a)

In [ ]:
tst = 'xPyTzPiTTyPzPPyTiTTz'
operator_from_string(tst)

In [ ]:
rhs=operator_from_string('iTyTTzTx')
i=np.identity(2, dtype=np.complex128)
c=np.kron(i,y)
d=np.kron(z,x)
rhs_real=np.kron(c,d)
diff = np.max(np.abs(rhs-rhs_real))
print(diff)

In [ ]:
out=operator_from_string('xTzPPzTyTTxTz')
np.shape(out)

In [ ]:
a=np.kron(x,z) + np.kron(z,y)
b=np.kron(x,z)
out_actual=np.kron(a,b)


In [ ]:
np.max(np.abs(out-out_actual))

In [ ]:
compute_p('xPy')

In [ ]:
compute_t('xPyTx')

In [ ]:
def compute_2p(inp):
    #print("Computing (2p)", inp)
    to_add = inp.split("PP")
    running_sum = empty_array_of_same_dim(inp)
    for i in range(len(to_add)):
        rhs = compute_1t(to_add[i])
        print("Adding \n", running_sum, "\n TO string ", to_add[i], " \n", rhs)
        running_sum += rhs
    
    #print("RESULT: ", inp, ": \n", running_sum)
    return running_sum

In [ ]:
def compute_2t(inp):
#    print("Computing (2t)", inp)
    to_tens = inp.split("TT")
    running_tens_prod=compute_2p(to_tens[0])
    for i in range(1,len(to_tens)):

 #       print("tensor product bw \n", running_tens_prod, "\n AND ", to_tens[i], " \n", compute_2p(to_tens[i]))
        running_tens_prod = np.kron(running_tens_prod, compute_2p(to_tens[i]))
  #  print("RESULT (2t) [shape : ", np.shape(running_tens_prod), "] \n",running_tens_prod )
    return running_tens_prod        

In [ ]:
compute_t('xTy', 1)

In [ ]:
to=compute_2p('xPzTxPPxTx')

In [ ]:
out1=compute_2t('xTyTTyTz')

In [ ]:
xty = np.kron(evo.sigmax(), evo.sigmay())
ytz = np.kron(evo.sigmay(), evo.sigmaz())
out = np.kron(xty, ytz)

In [ ]:
np.max(np.abs(out - out1))

In [ ]:
t_str, p_str, max_t, max_p = get_t_p_strings(name)
constit_names = name.split(p_str)
ind_ops = list()

for i in range(len(constit_names)):
    ind_ops.append(constit_names[i].split(t_str))
op_list = list()
for j in range(len(ind_ops)):
    running_sum = evo.sigmax() - evo.sigmax()
    print("op ", j)
    for k in range(len(ind_ops[j])):
        print("k=", k)
        op_name = ind_ops[j][k][0]
        op = paulis_list[op_name]
        running_sum += op
    op_list.append(running_sum)


running_kron_prod = op_list[0]
for j in range(1,len(op_list)):
    running_kron_prod = np.kron(running_kron_prod, op_list[j])
        